# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846797269822                   -0.70    4.5
  2   -7.852293041860       -2.26       -1.53    1.0
  3   -7.852646037380       -3.45       -2.52    3.2
  4   -7.852646677405       -6.19       -3.37    2.2
  5   -7.852646685830       -8.07       -4.75    1.2
  6   -7.852646686723       -9.05       -5.17    4.0
  7   -7.852646686729      -11.16       -5.98    1.5
  8   -7.852646686730      -12.45       -7.55    2.2
  9   -7.852646686730      -14.35       -7.46    2.2
 10   -7.852646686730   +  -14.57       -8.91    2.2
 11   -7.852646686730      -14.75       -9.57    2.5
 12   -7.852646686730   +    -Inf       -9.64    1.0
 13   -7.852646686730   +    -Inf      -10.99    1.8
 14   -7.852646686730   +  -14.75      -11.35    2.2
 15   -7.852646686730      -14.57      -11.66    1.2
 16   -7.852646686730   +  -15.05      -12.34    2.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846812705493                   -0.70           4.5
  2   -7.852526898749       -2.24       -1.63   0.80    2.0
  3   -7.852636434585       -3.96       -2.71   0.80    1.0
  4   -7.852646450428       -5.00       -3.28   0.80    2.2
  5   -7.852646674792       -6.65       -4.10   0.80    1.2
  6   -7.852646686360       -7.94       -4.73   0.80    1.5
  7   -7.852646686723       -9.44       -5.57   0.80    2.0
  8   -7.852646686730      -11.20       -6.49   0.80    1.8
  9   -7.852646686730      -12.49       -7.44   0.80    2.2
 10   -7.852646686730      -14.21       -8.17   0.80    2.0
 11   -7.852646686730   +  -14.75       -9.08   0.80    2.0
 12   -7.852646686730      -14.75       -9.58   0.80    2.0
 13   -7.852646686730      -14.75      -10.57   0.80    1.2
 14   -7.852646686730   +  -14.57      -11.02   0.80    2.8
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.396813e+01     3.542703e+00
 * time: 0.4143180847167969
     1     1.284169e+00     1.844049e+00
 * time: 0.6115701198577881
     2    -1.621814e+00     2.051491e+00
 * time: 0.6346549987792969
     3    -3.640671e+00     2.003165e+00
 * time: 0.6675131320953369
     4    -4.891558e+00     1.833512e+00
 * time: 0.7009930610656738
     5    -6.636833e+00     1.159500e+00
 * time: 0.7342691421508789
     6    -7.370333e+00     6.969575e-01
 * time: 0.7671611309051514
     7    -7.616082e+00     5.227928e-01
 * time: 0.7904541492462158
     8    -7.759701e+00     1.034338e-01
 * time: 0.8134291172027588
     9    -7.796090e+00     1.483712e-01
 * time: 0.8363480567932129
    10    -7.824566e+00     5.586510e-02
 * time: 0.859483003616333
    11    -7.837204e+00     4.412270e-02
 * time: 0.8825860023498535
    12    -7.843677e+00     3.827675e-02
 * time: 0.9054319858551025
    13    -7.845437e+00     4.439880e-02
 * time: 0.92820596694

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846817668890                   -0.70    4.5
  2   -7.852306301511       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686669                   -2.55
  2   -7.852646686730      -10.21       -5.89
  3   -7.852646686730   +  -14.75      -12.21


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.404425270533604e-13
|ρ_newton - ρ_scfv| = 1.4078050139936343e-13
|ρ_newton - ρ_dm|   = 4.3151583913869374e-10
